Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [28]:
import functools
from dataclasses import dataclass



import numpy as np



from tqdm.auto import tqdm



from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [6]:
UNIVERSE_SIZE = 10
NUM_SETS = 5
DENSITY = 0.3

In [7]:
# DON'T EDIT THESE LINES!

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)


def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


@counter
def cost(solution):
    """Returns the cost of a solution (to be minimized) tracking number of calls"""
    return COSTS[solution].sum()

# Squillero's EA

## Helper Functions

In [80]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def num_covered(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.sum(np.logical_or.reduce(SETS[solution]))


@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None


def fitness(individual):
    return int(num_covered(individual.genome)), -float(cost(individual.genome))


def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def xover(p1: Individual, p2: Individual):
    m = np.random.rand(NUM_SETS) < 0.5
    genome = p1.genome.copy()
    genome[m] = p2.genome[m]
    return Individual(genome)

## Have Fun!

In [81]:
POPULATION_SIZE = 10
population = [Individual(np.random.rand(NUM_SETS) < 0.5) for _ in range(POPULATION_SIZE)]
for i in population:
    i.fitness = fitness(i)

OFFSPRING_SIZE = 4

offspring = list()
for _ in range(OFFSPRING_SIZE):
    i1 = parent_selection(population)
    i2 = parent_selection(population)
    o = xover(i1, i2)
    offspring.append(o)

for i in offspring:
    i.fitness = fitness(i)

population.extend(offspring)
population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]

In [82]:
population

[Individual(genome=array([False,  True,  True,  True, False]), fitness=(10, -12.611435060500822)),
 Individual(genome=array([False,  True,  True,  True,  True]), fitness=(10, -15.959804582602537)),
 Individual(genome=array([False,  True,  True, False,  True]), fitness=(8, -10.08670986716244)),
 Individual(genome=array([ True, False, False,  True, False]), fitness=(7, -9.22146423754181)),
 Individual(genome=array([False,  True,  True, False, False]), fitness=(6, -6.738340345060727)),
 Individual(genome=array([False,  True, False, False,  True]), fitness=(6, -7.943162942089854)),
 Individual(genome=array([False,  True, False, False,  True]), fitness=(6, -7.943162942089854)),
 Individual(genome=array([False,  True, False, False,  True]), fitness=(6, -7.943162942089854)),
 Individual(genome=array([False, False, False,  True,  True]), fitness=(6, -9.22146423754181)),
 Individual(genome=array([False, False, False,  True,  True]), fitness=(6, -9.22146423754181))]

In [70]:
offspring

[Individual(genome=array([ True, False, False, False, False]), fitness=np.float64(-3.348369522101714)),
 Individual(genome=array([ True, False, False, False, False]), fitness=np.float64(-3.348369522101714)),
 Individual(genome=array([False, False, False, False, False]), fitness=np.float64(-0.0)),
 Individual(genome=array([ True, False,  True, False, False]), fitness=np.float64(-5.4919164471743))]

In [23]:
fitness(population[0])

np.float64(-14.71338068471611)

In [61]:
population[0].genome

array([False,  True, False, False, False])

In [64]:
population[0].genome[np.array([False, True, False, False, False])] = False

In [65]:
population[0].genome

array([False, False, False, False, False])